In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
root_path = '../'
sys.path.insert(0, root_path)

In [ ]:
import os
import pandas as pd
from natsort import natsorted

from scripts.create_problem import load_problem_set
from scripts.run_solver import load_results
from utils import *

In [ ]:
# read_dir = '../data/test'
read_dir = "../data/normal_10"
# read_dir = "../data/various"
# read_dir = "../data/normal"
# read_dir = "../data/distributions"
problem_list = natsorted(os.listdir(read_dir))

# problem_list = [
#     'SPP',
#     'GraphColoring',
#     'Knapsack',
#     'MVC',
#     'QUBO',
#     'SetPack',
#     'WMaxCut',
#     'NumberPartitioning',
#     'MaxCut',
#     'WMVC',
#     'Ising',
#     'Max2Sat',
#     'QAP'
# ]

X, y = get_Xy(read_dir, problem_list, suffix='test')
X_tsne = get_tsne(X)
show_tsne(problem_list, X_tsne, y)
print_unique(read_dir, problem_list)

In [ ]:
# from scripts.create_problem import load_problem_set
# from scripts.run_solver import load_results

# d_x = []
# problem_name = 'QAP'
# problems = load_problem_set(f'{read_dir}/{problem_name}/train')
# for problem in problems:
#     d_x.append(problem.info['x_best'])
#     print(problem.Q)
# x_unique = torch.unique(torch.stack(d_x), dim=0)
# print(x_unique)

In [ ]:
# read_dir = '../results/test'
read_dir = '../results/normal_10'
# read_dir = '../results/various'
# read_dir = '../results/normal'
# read_dir = '../results/distributions'

problem_list = natsorted(os.listdir(read_dir))
meta_results = defaultdict(dict)
meta_stats = defaultdict(dict)
for problem in problem_list:
    for solver in ('BRUTE',): #('RandomSearch', 'PSO', 'PROTES'):
        meta_results[problem][solver] = get_meta_results(problem, solver, read_dir, suffix='test', budget=1024, n_steps=100)
        meta_stats[problem][solver] = get_trajectory_stats(problem, solver, read_dir, suffix='test')

In [ ]:
show_meta_results(meta_results)
# flattened = {
#     (row_key, col_key): subdict
#     for row_key, cols in meta_stats.items()
#     for col_key, subdict in cols.items()
# }
# pd.DataFrame.from_dict(flattened, orient='index')

In [ ]:
title = 'Normal'
problem_averaged_meta_results = get_problem_averaged_meta_dict(meta_results, title)
problem_averaged_meta_stats = get_problem_averaged_meta_dict(meta_stats, title)

show_meta_results(problem_averaged_meta_results)
pd.DataFrame(problem_averaged_meta_stats[title]).T.round(2)

In [ ]:
problems = load_problem_set('../data/normal_10/Normal(0, 1)/test')

In [ ]:
for problem in problems:
    print(problem.info['x_best'], problem.info['y_best'])

In [ ]:
from solvers.brute import get_grid

In [ ]:
problem = problems[0]
x = get_grid(problem.d, problem.n)
y = problem.target(x)

In [ ]:
torch.all(x @ torch.pow(problem.n, torch.arange(problem.d).flip(0)) == torch.arange(problem.n ** problem.d))